In [54]:
# retrieve plasmid sequence from GenBank
from pydna.genbank import Genbank
from Bio import SeqIO
from Bio.Restriction import AllEnzymes
from pydna.parsers import parse_snapgene
import matplotlib.pyplot as plt
import numpy as np
import itertools

from snapgene_reader import snapgene_file_to_dict, snapgene_file_to_seqrecord

In [55]:
from pydna.dseqrecord import Dseqrecord
from pydna.design import primer_design

In [56]:
def load_from_genebank(email, sequence):
    gb = Genbank(users_email=email)  
    seq = gb.nucleotide(sequence)   
    return seq

In [57]:
email = ""

In [58]:
# Some gene sequence
seqs_id = ["M61151.1", "X51424.1", "PV006683.1", "PV006682.1", "OR455449.1"]
# Create a Dseq object
seqs = []
for seq in seqs_id:
    seqs.append(load_from_genebank(email, seq))

genes = list(map(lambda seq: Dseqrecord(seq), seqs))

In [59]:
# Define primer length (typical range 18-22 nt)
primer_length = 20

In [60]:
# Forward primer: first 20 nt
fwd_primers = list(map(lambda gene: gene[:primer_length], genes))

In [61]:
# Reverse primer: last 20 nt, reverse complement
rev_primers = list(map(lambda gene: gene[-primer_length:].rc(), genes))

In [62]:
for fwd_primer, rev_primer in zip(fwd_primers, rev_primers):
    print("Forward primer:", fwd_primer)
    print("Reverse primer:", rev_primer)

Forward primer: Dseqrecord
circular: False
size: 20
ID: part_M611511
Name: part_ATUAUX
Description: Agrobacterium rhizogenes trytophan oxygenase (aux1) and hydrolase (aux2) genes, complete cds
Number of features: 0
/molecule_type=DNA
/topology=linear
/data_file_division=BCT
/date=23-AUG-1993
/accessions=['M61151', 'J03688']
/sequence_version=1
/keywords=['aux1 gene', 'aux2 gene', 'hydrolase', 'tryptophan oxygenase']
/source=Rhizobium rhizogenes (Agrobacterium rhizogenes)
/organism=Rhizobium rhizogenes
/taxonomy=['Bacteria', 'Pseudomonadati', 'Pseudomonadota', 'Alphaproteobacteria', 'Hyphomicrobiales', 'Rhizobiaceae', 'Rhizobium/Agrobacterium group', 'Rhizobium']
/references=[Reference(title='The TR.DNA region carrying the auxin synthesis genes of the Agrobacterium rhizogenes agropine-type pRiA4: Nucleotide sequence analysis and introduction into tobacco plants', ...)]
Dseq(-20)
GTCGACAGTCGCAACAGCAA
CAGCTGTCAGCGTTGTCGTT
Reverse primer: Dseqrecord
circular: False
size: 20
ID: part_M61151

In [32]:
gene

Dseqrecord(-41)

In [43]:
# Or use Primer3 for more sophisticated design (optional):
gene
primers = primer_design(gene)
# Design primers with default parameters (Tm, GC content, etc.)
# primers = primer3.design_primers()
print(primers.forward_primer.seq)
print(primers.reverse_primer.seq)
# print("Primer3 forward primer:", primers['FWDPRIMER'])
# print("Primer3 reverse primer:", primers['REVPRIMER'])

ATGCGTACGTTAGCTAG
TAGCTAGCTAGCTACGC
